In [10]:
! pip install mediapipe
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import load_model
import pandas as pd

# Load pre-trained deep learning model for rule violations
detection_model = pd.read_csv("/content/Squash game.csv")

# Initialize MediaPipe for player pose estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# Initialize OpenCV video capture
cap = cv2.VideoCapture("squash_match.mp4")

# Ball tracking parameters
ball_lower = np.array([30, 150, 50])  # HSV lower bound
ball_upper = np.array([255, 255, 180])  # HSV upper bound

# Court boundaries (example values, adjust as needed)
court_top, court_bottom = 50, 500
court_left, court_right = 50, 700

def test_violation_detection():
    """Function to test rule-based violation detection."""
    test_cases = [
        ((40, 300), "Fault - Ball Out"),  # Out-of-bounds (left side)
        ((750, 300), "Fault - Ball Out"),  # Out-of-bounds (right side)
        ((400, 600), "Fault - Ball Out"),  # Out-of-bounds (bottom side)
        ((400, 100), "Play On"),  # In bounds
        ((400, 300), "Let - Possible Interference")  # Near a player
    ]

    for ball_position, expected_decision in test_cases:
        decision = "Play On"
        x, y = ball_position
        if x < court_left or x > court_right or y < court_top or y > court_bottom:
            decision = "Fault - Ball Out"

        for px, py in [(400, 300)]:  # Simulated player position
            if abs(px - x) < 50 and abs(py - y) < 50:
                decision = "Let - Possible Interference"

        assert decision == expected_decision, f"Test failed for {ball_position}: Expected {expected_decision}, Got {decision}"

    print("All rule-based violation detection tests passed!")

test_violation_detection()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, ball_lower, ball_upper)

    # Find ball contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    ball_position = None
    for contour in contours:
        if cv2.contourArea(contour) > 100:  # Threshold to avoid noise
            (x, y), radius = cv2.minEnclosingCircle(contour)
            center = (int(x), int(y))
            ball_position = center
            cv2.circle(frame, center, int(radius), (0, 255, 0), 2)

    # Player detection using MediaPipe Pose
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)
    player_positions = []
    if result.pose_landmarks:
        for landmark in result.pose_landmarks.landmark:
            player_positions.append((int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])))
        mp.solutions.drawing_utils.draw_landmarks(
            frame, result.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # Rule-Based Violation Detection
    decision = "Play On"
    if ball_position:
        x, y = ball_position
        # Check if the ball is out-of-bounds
        if x < court_left or x > court_right or y < court_top or y > court_bottom:
            decision = "Fault - Ball Out"

        # Check for interference (basic check: if a player is too close to the ball)
        for px, py in player_positions:
            if abs(px - x) < 50 and abs(py - y) < 50:
                decision = "Let - Possible Interference"

        # Deep learning model for advanced rule classification
        if ball_position and player_positions:
            ball_x, ball_y = ball_position
            player_x, player_y = player_positions[0]  # Take the first player as an example

            # Create a feature vector: ball position + player position (make sure it has 10 features)
            feature_vector = np.array([ball_x, ball_y, player_x, player_y,
                                       abs(ball_x - player_x), abs(ball_y - player_y)])

            # Ensure the feature vector has the correct shape (1, 10)
            feature_vector = np.pad(feature_vector, (0, 10 - len(feature_vector)), mode='constant')  # Padding if needed

            # Predict using the model
            violation_prediction = detection_model.predict(feature_vector.reshape(1, -1))  # Reshaping to (1, 10)
            if violation_prediction[0][0] > 0.5:
                decision = "Stroke"

    # Display decision on screen
    cv2.putText(frame, f"Decision: {decision}", (50, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show processed frame
    cv2.imshow("Squash Referee System", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


All rule-based violation detection tests passed!
